# Active Ind Testing

In [1]:
import numpy as np
import gurobipy as gp
import math
import sympy
import contextlib
import time
import os
from utils import result, METHODS, INF, EPS
from mps_reader_preprocessor import read_mps_preprocess
from polyhedral_model import PolyhedralModel
from polyhedron import Polyhedron

## Read in Polyhedron P

In [2]:
problem_dir = 'netlib_lp_subset'
# problem = 'kb2'
# problem = 'adlittle'
# problem = 'angela_test_prob'
# problem = 'network_ex'
# problem = 'gfrd-pnc'
# problem = 'modszk1'
# problem = 'cycle'
# problem = 'cre-c'
# problem = 'scsd8'
# problem = 'bandm'
problem = 'qap08'
# problem = 'scsd1'
# problem = 'scsd6'
mps_fn=os.path.join(problem_dir, problem)
results_dir='results'
max_time=300
sd_method='dual_simplex'
reset=False,
partition_polytope=False
n=0
k=0
spindle=False
spindle_dim=0
n_cone_facets=0
n_parallel_facets=0

## Perform SDCA

In [3]:
### Build Initial Polyhedron from file
print(f'Reading {mps_fn}...')
c, B, d, A, b = read_mps_preprocess(mps_fn)
print('Building polyhedron...')
P = Polyhedron(B, d, A, b, c)
rank_A = np.linalg.matrix_rank(A)

print('Finding feasible solution...')
x_feasible, vbasis, cbasis = P.find_feasible_solution(verbose=False)

x = x_feasible
c=None
verbose=False
method='dual_simplex'
reset=False
max_time=300
first_warm_start=None
save_first_steps=0
problem_name=''

if c is not None:
    P.set_objective(c)

t0 = time.time()
x_current = x
if save_first_steps:
    np.save('solutions/{}_0.npy'.format(problem_name), x_current)      
active_inds = P.get_active_constraints(x_current)
    
pm = P.build_polyhedral_model(active_inds=active_inds, method=method)

if first_warm_start is not None:
    print('Using custom warm start')
    pm.set_solution(first_warm_start)
t1 = time.time()
build_time = t1 - t0
print('Polyhedral model build time: {}'.format(build_time))
    
sub_times = {'sd': [], 'step': [], 'solve': [], 'phase_times': []}
active_inds_it = []
descent_circuits = []
obj_values = []
step_sizes = []
iter_times = []
simplex_iters = []
iteration = 0
active_inds_it.append(active_inds)
obj_value = P.c.dot(x_current)
obj_values.append(obj_value)
t2 = time.time()
iter_times.append(t2 - t1)


# compute steepest-descent direction
descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(verbose=verbose)
simplex_iters.append(num_steps)
sub_times['solve'].append(solve_time)
sub_times['phase_times'].append(phase_times)
    
t3 = time.time()
sub_times['sd'].append(t3 - t2)
simp_switches = []
simp_switches.append(False)
    
while abs(steepness) > EPS:
        
    t3 = time.time()
    if reset:
        pm.reset()
        
    # take maximal step
    x_current, alpha, active_inds = P.take_maximal_step(descent_direction, y_pos, y_neg)

    for i in x_current:
        if i<0:
            print(i)

    if len(active_inds) < (P.n - rank_A):
        simp_switches.append(True)
    else:
        simp_switches.append(False)
        
    if iteration % 50 == 0 or iteration == 1:
        print('\nIteration {}'.format(iteration))
        print('Objective: {}'.format(obj_value))
        print('Steepness: {}'.format(steepness))
        print('Step length: {}'.format(alpha))
        # print('Descent Direction: {}'.format(descent_direction))
        
    t4 = time.time()
    active_inds_it.append(active_inds)
    obj_value = P.c.dot(x_current)
    obj_values.append(obj_value)
    iter_times.append(t4 - t1)
    sub_times['step'].append(t4 - t3) 
    descent_circuits.append(descent_direction)
    step_sizes.append(alpha)     
                
    if math.isinf(alpha):
        # problem is unbounded
        result(status=1, circuits=descent_circuits, steps=step_sizes)

    print(f'Number of Active Inds for Iteration {iteration}: {len(active_inds)}')
    print(f'Switch to Facet Restriction for Iteration {iteration}: {simp_switches[-1]}')
        
    # compute steepest-descent direction
    if simp_switches[-1] and not simp_switches[-2]:
        init_inds = active_inds
        print(f'initial inds for switch: {init_inds}')

    if simp_switches[-1]:
        print('using init_inds')
        pm.set_active_inds(init_inds, simp_switch = simp_switches[-1])
    elif simp_switches[-2] and not simp_switches[-1]:
        ##### finish using primal simplex 
        # not to self: when I used the set_solution method, it said that the x_current at the time of the switch was not feasible
        print('switched to primal simplex')
        simp_result = P.solve_lp_ws(x_current,verbose=False, record_objs=True)
        print(simp_result)
        break
    else:
        print('using active_inds')
        pm.set_active_inds(active_inds, simp_switch = simp_switches[-1])
    
    print('---------------------------------------------------------------')
    
    descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(verbose=verbose)
        
    t5 = time.time()
    sub_times['sd'].append(t5 - t4)
    sub_times['solve'].append(solve_time)
    sub_times['phase_times'].append(phase_times)
    simplex_iters.append(num_steps)
        
    iteration += 1
    current_time = t5 - t1
    if current_time > max_time:
        result(status=2)
    if iteration <= save_first_steps:
        np.save('solutions/{}_{}.npy'.format(problem_name, iteration), x_current)

t6 = time.time()
total_time = t6 - t1   
print('Total time for steepest-descent scheme: {}'.format(total_time))

Reading netlib_lp_subset\qap08...
Building polyhedron...
Problem size: n = 1632,  m_B = 1632,  m_A = 912
Finding feasible solution...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-01
Building polyhedral model. Solve method: dual_simplex
Set parameter Method to value 1
Polyhedral model built!
Polyhedral model build time: 2.7586863040924072

Iteration 0
Objective: 303.6065924991011
Steepness: -7.500000000000017
Step length: 0.42444896301039126
Number of Active Inds for Iteration 0: 878
Switch to Facet Restriction for Iteration 0: True
initial inds for switch: [64, 66, 68, 70, 71, 72, 75, 76, 77, 79, 81, 83, 84, 87, 88, 89, 90, 91, 92, 94, 97, 98, 99, 100, 102, 103, 107, 108, 109, 110, 111, 113, 114, 116, 118, 119, 120, 122, 123, 124, 125, 127, 128, 129, 132, 135, 137, 138, 140, 142, 143, 145, 147, 148, 151, 153, 156, 157, 159, 161, 162, 163, 164, 171, 172, 173, 175, 177, 179, 180, 181, 182, 183, 184, 185, 188, 190, 192, 193, 194, 195, 197, 198, 2

In [9]:
obj_values[0]

303.6065924991011

In [4]:
print(P.n)
print(rank_A)
print(P.m_B)
print(math.floor(P.n/10))

1632
742
1632
163


In [5]:
print(obj_values[-1])
# print(x_current)
# print(simp_result)


293.60086667760174


In [6]:
print(f'Reading {mps_fn}...')
c, B, d, A, b = read_mps_preprocess(mps_fn)
print('Building polyhedron...')
P = Polyhedron(B, d, A, b, c)
print('\nSolving with simplex method...')
lp_result = P.solve_lp(verbose=False, record_objs=True)
print('\nSolution using simplex method:')
print(lp_result)

Reading netlib_lp_subset\qap08...
Building polyhedron...
Problem size: n = 1632,  m_B = 1632,  m_A = 912

Solving with simplex method...

Solution using simplex method:

Optimal objective: 203.5000000000028
Total solve time: 0.046000003814697266
Number of iterations: 461.0


In [7]:
equality = np.all(np.isclose(np.dot(A, np.array(x_current)), b))
inequality = np.all(np.dot(B, np.array(x_current)) <= d)
print(equality)
print(inequality)

True
False


In [8]:
# P.bfs_soln_chck(x_current)
P.set_solution(x_current)


set solution is not feasible


RuntimeError: Failed to find feasible solution.

In [ ]:
# print(np.count_nonzero(np.array(x_current)))
for i in x_current:
    if i<0:
        print(i)